In [25]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time 
import random

In [26]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/104rrc/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


In [27]:
# url_list = ['https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784118/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=78704&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d2412#listing=333651233/NONE',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED']

url_df = pd.read_csv('../Resources/all_urls_list.csv')
url_list = url_df['urls']
url_list

0       https://www.cargurus.com/Cars/new/searchresult...
1       https://www.cargurus.com/Cars/new/searchresult...
2       https://www.cargurus.com/Cars/new/searchresult...
3       https://www.cargurus.com/Cars/new/searchresult...
4       https://www.cargurus.com/Cars/new/searchresult...
                              ...                        
2216    https://www.cargurus.com/Cars/new/searchresult...
2217    https://www.cargurus.com/Cars/new/searchresult...
2218    https://www.cargurus.com/Cars/new/searchresult...
2219    https://www.cargurus.com/Cars/new/searchresult...
2220    https://www.cargurus.com/Cars/new/searchresult...
Name: urls, Length: 2221, dtype: object

In [28]:
cars_df = pd.DataFrame({})


In [29]:
browser.visit(url_list[0])

In [30]:
failed_tries = []

In [31]:
start = 1200
end = 1400
# start with just 1 loop
for i in range(start,end):
    browser.visit(url_list[i])
    time.sleep(random.randint(3,6))
    
    # place into try / except block:
    try:
        print(f"Adding {i} of {end}")
        # scrape using find by tag (section).  Should return list.
        keys = browser.find_by_tag('dl').find_by_tag('dt')
        values = browser.find_by_tag('dl').find_by_tag('dd')
        print('.', end='')
        # loop through section list and find the one that has "Dealer's Price"
#         section_list = []
#         for e in elems:
#             section_list.append(e.html)
        # extract section list item into section variable

        # run replaces on tags in section to turn it into table
        

        # put table into single dataframe temp_df (NOT scrape_cars_df)
        temp_df = pd.DataFrame({})
        for j in range(len(keys)):
            if values[j].text == 'Contact the dealer for pricing details':
                values.pop(j)
            temp_df[keys[j].text]= [values[j].text]
        temp_df
        print('.', end='')
        # create search string to find remaining car data
        # search_str = " - " + temp_df["Dealer's Price:"][0]
        # find <h1> tag which contains search_str
        search_str = "IpF2YF"
        title = browser.find_by_tag('h1')
        price = title[0].html.split(' ')[-1]
        model = title[0].html.split(' ')[2]
        
        temp_df['Price:'] = [price]
        temp_df['Model:'] = [model]
        print('.', end='')
        # split element string by space
        # year = e[0]
        # make = e[1]
        # model = e[2]
        dealernames = browser.find_by_tag('a')
        for d in dealernames:
            if d['data-cg-ft'] == 'vdp-dealer-cg-homepage-link':
                dealername = d.text
        temp_df['Dealer Name:'] = [dealername]
        # add columns and data to dataframe (temp_df['year'] = [year])
        print('.', end='')
        
        dealeraddress = browser.find_by_tag('span')
        for d in dealeraddress:
            if d['data-track-ui'] == 'dealer-address':
                dealeradd = d.text
        dealeradd = dealeradd.replace('\n', '<br>')        
        temp_df["Dealer Address:"] = [dealeradd]
        # append new dataframe to scrape_cars_df if successful
        print('.', end='')
        
        cars_df = pd.concat([cars_df, temp_df], axis=0, ignore_index=True)
        print(' Success!')
        
    
    # except: add i into failed list
    except:
        failed_tries.append(i)
        print(' Error found.')
cars_df.to_csv(f'../Resources/{start}-{end}.csv')

Adding 1200 of 1400
..... Success!
Adding 1201 of 1400
 Error found.
Adding 1202 of 1400
 Error found.
Adding 1203 of 1400
..... Success!
Adding 1204 of 1400
..... Success!
Adding 1205 of 1400
..... Success!
Adding 1206 of 1400
..... Success!
Adding 1207 of 1400
..... Success!
Adding 1208 of 1400
..... Success!
Adding 1209 of 1400
 Error found.
Adding 1210 of 1400
 Error found.
Adding 1211 of 1400
..... Success!
Adding 1212 of 1400
 Error found.
Adding 1213 of 1400
..... Success!
Adding 1214 of 1400
 Error found.
Adding 1215 of 1400
..... Success!
Adding 1216 of 1400
..... Success!
Adding 1217 of 1400
..... Success!
Adding 1218 of 1400
..... Success!
Adding 1219 of 1400
..... Success!
Adding 1220 of 1400
 Error found.
Adding 1221 of 1400
..... Success!
Adding 1222 of 1400
..... Success!
Adding 1223 of 1400
..... Success!
Adding 1224 of 1400
..... Success!
Adding 1225 of 1400
..... Success!
Adding 1226 of 1400
..... Success!
Adding 1227 of 1400
 Error found.
Adding 1228 of 1400
..... Su

In [32]:
dealeraddress = browser.find_by_tag('span')
for d in dealeraddress:
    if d['data-track-ui'] == 'dealer-address':
        dealeradd = d.text
dealeradd = dealeradd.replace('\n', '<br>')  
dealeradd
#         temp_df["Dealer Address:"] = [dealeradd]

'3777 Gerstner Memorial Blvd<br>Lake Charles, LA 70607'

In [33]:
keys = browser.find_by_tag('dl').find_by_tag('dt')
values = browser.find_by_tag('dl').find_by_tag('dd')
temp_df = pd.DataFrame({})


for j in range(len(keys)):
    if values[j].text == 'Contact the dealer for pricing details':
        values.pop(j)
    temp_df[keys[j].text]= [values[j].text]
temp_df

# values[2].text

,Make:,Model:,Year:,Trim:,Body type:,Exterior color:,Interior color:,VIN:,Stock number:
0,Jeep,Wrangler Unlimited 4xe,2022,Rubicon 4WD,SUV / crossover,Silver Zynith Clearcoat,Black,1C4JJXR62NW191894,NW191894


In [34]:
cars_df

,Make:,Model:,Year:,Trim:,Body type:,Exterior color:,Interior color:,Mileage:,VIN:,Stock number:,Price:,Dealer Name:,Dealer Address:
0,Jeep,Wrangler,2022,Sport 4WD,SUV / crossover,Brown (Limited Edition Gobi Clearcoat),Black,0 mi,1C4HJXDN6NW243959,W243959,"$40,359",San Antonio Dodge Chrysler Jeep Ram,"11910 N Ih 35<br>San Antonio, TX 78233"
1,Jeep,Wrangler,2022,Willys 4WD,SUV / crossover,Bright White,Black,0 mi,1C4HJXDNXNW253569,253569,"$42,125",Mac Haik Chrysler Dodge Jeep RAM Energy Corridor,"12111 Katy Fwy<br>Houston, TX 77079"
2,Jeep,Wrangler,2022,Sport Altitude 4WD,SUV / crossover,Gray,Black,0 mi,1C4HJXDN8NW242280,53885325,"$43,379",Frisco Chrysler Dodge Jeep Ram,"9640 State Highway 121<br>Frisco, TX 75035"
3,Jeep,Wrangler,2022,Sport Altitude 4WD,SUV / crossover,Black,Black,0 mi,1C4HJXDG9NW226056,53885323,"$43,879",Frisco Chrysler Dodge Jeep Ram,"9640 State Highway 121<br>Frisco, TX 75035"
4,Jeep,Wrangler,2022,Sport S 4WD,SUV / crossover,Sting-Gray Clearcoat,Black,7 mi,1C4HJXDG8NW241924,J18093,"$44,575",Covert Chrysler Dodge Jeep Ram-Austin,"8107 Research Blvd<br>Austin, TX 78758"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Jeep,Wrangler,2022,Rubicon 4WD,SUV / crossover,Sting-Gray Clearcoat,Black,NaN,1C4JJXR66NW233340,NW233340,"$67,114",Mark Dodge,"3777 Gerstner Memorial Blvd<br>Lake Charles, L..."
166,Jeep,Wrangler,2022,Rubicon 4WD,SUV / crossover,Sting-Gray Clearcoat,Black,NaN,1C4JJXR62NW259840,NW259840,"$67,137",Mark Dodge,"3777 Gerstner Memorial Blvd<br>Lake Charles, L..."
167,Jeep,Wrangler,2022,Rubicon 4WD,SUV / crossover,Bright White Clearcoat,Black,NaN,1C4JJXR65NW183000,NW183000,"$67,209",Mark Dodge,"3777 Gerstner Memorial Blvd<br>Lake Charles, L..."
168,Jeep,Wrangler,2022,Sahara 4WD,SUV / crossover,Bright White Clearcoat,Black,10 mi,1C4JJXP61NW121418,NW121418,"$67,245",Nyle Maxwell CDJR Austin,"13817 U.S. 183<br>Austin, TX 78750"


In [35]:
len(failed_tries)

30

In [36]:
search_str = "IpF2YF"
title = browser.find_by_tag('h1')
title[0].html.split(' ')[-1]

'$67,329'